In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import random
from pandas import DataFrame
from datetime import datetime

# input file path
workdir = "D://DrugResponsivenessGuideline_log//Github//200914_reupload//Dataset//Scenario4_EC-11K"

In [2]:
# read input file and index file for training and test set
dataset = np.load(workdir + "//200914_EC11K.npz")
ss0 = np.load(workdir + '//200914_EC11K_r0_9_1.npz')

In [3]:
# input and index file assign
x = dataset['x']
y = dataset['y']
# y_linear = dataset['y_lnIC50']
ss0_train = ss0['train']
ss0_test = ss0['test']

In [4]:
# training and test dataset assign
training_image_array, training_label_array = x[ss0_train], y[ss0_train]
test_image_array, test_label_array = x[ss0_test], y[ss0_test]


In [5]:
# checking the size of dataset
x.shape

(11360, 18988)

In [6]:
# checking the size of training set
training_image_array.shape

(10224, 18988)

In [7]:
train_X, train_y, test_X, test_y = training_image_array, training_label_array, test_image_array, test_label_array


In [8]:
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(10224, 18988) (10224,) (1136, 18988) (1136,)


In [9]:
# impute nan to zero
train_X = np.nan_to_num(train_X)
test_X = np.nan_to_num(test_X)

In [10]:
# data typing and the number of dataset
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
# train_X /= 255
# test_X /= 255
print('train_X shape:', train_X.shape)
print(train_X.shape[0], 'train samples')
print(test_X.shape[0], 'test samples')

train_X shape: (10224, 18988)
10224 train samples
1136 test samples


In [11]:
# model fitting
StartTime8 = datetime.now()
print("StartTime :", StartTime8)
svr_lin = SVR(C=0.01)
y_lin = svr_lin.fit(train_X, train_y).predict(test_X)
EndTime8 = datetime.now()
print("EndTime :", EndTime8)

StartTime : 2020-09-17 10:33:48.941449
EndTime : 2020-09-17 11:10:02.099805


In [12]:
import matplotlib
from matplotlib import pyplot as plt

In [13]:
# saving predicted ln(IC50)s and ln(IC50)s in the test set
a = pd.DataFrame(y_lin)
b = pd.DataFrame(test_label_array)
c = pd.concat([a,b], axis=1)
c.columns=["Predicted","Test"]

workdir = "D://DrugResponsivenessGuideline_log//Github//200914_reupload//Result//EC-11K"
c.to_csv(workdir + '//200915_EC11K_SVR_pred_and_val_result.csv', index=False)


In [14]:
# result table
c.head()

,Predicted,Test
0,1.962280,-0.940726
1,1.979780,2.079442
2,1.975818,2.079442
3,1.969292,2.079442
4,1.969669,2.079442


In [15]:
# the length of result table
y_lin.shape

(1136,)

In [16]:
# model save
import pickle
from sklearn.externals import joblib
joblib.dump(svr_lin, workdir + '//200915_EC11K_SVR_model.pkl') 


C:\Users\ylab\AppData\Local\conda\conda\envs\ronny\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['D://DrugResponsivenessGuideline_log//Github//200914_reupload//Result//EC-11K//200915_EC11K_SVR_model.pkl']

In [17]:
# check whether the saved model is working
regr_from_joblib = joblib.load(workdir + '//200915_EC11K_SVR_model.pkl') 
regr_from_joblib.predict(test_X)

array([1.96227965, 1.97977981, 1.97581792, ..., 1.96477572, 1.94013618,
       1.97216821])

In [18]:
# plotting
plt.scatter(test_label_array,predicted_value,c='red')
plt.xlabel('test_IC50_value')
plt.ylabel('predicted_IC50_value')
plt.savefig(workdir + '//200915_EC11K_SVR_plot.png')

NameError: name 'predicted_value' is not defined

In [ ]:
# R-sqaured value
from sklearn.metrics import r2_score
r2_value = r2_score(b,a)
print(r2_value)

In [ ]:
# linear regression slope
from scipy.stats import linregress
linregress(b[0], a[0])

In [ ]:
# RMSE
rse = ((b[0]-a[0])**2).sum()
mse = rse / len(b)
print("Final rmse value is =",np.sqrt(mse))